# Import all packages/library.

In [239]:
import importlib
import helper
importlib.reload(helper)

<module 'helper' from 'c:\\Grace\\work\\`Apply\\CV_Portfolio\\Github\\3_Sales_Data_Forecasting\\helper.py'>

In [240]:
from helper import *
import pandas as pd

# Data Preprocessing

Read the raw data file.

In [241]:
raw_df = pd.read_csv('./data/retail_sales_synthetic.csv')
df = raw_df.copy()  # Copy to ensure every change made in this code doesn't affect the raw data.

In [242]:
# It will be truncated if we printed it as df.head() or df.describe() so we can't check (see) all columns.
# In order to avoid that, we need to print it partially.
n_col = len(df.columns)

In [243]:
print('\nInitial Dataframe Head:')
print(df.head().iloc[:, :int(n_col/2)])
print(df.head().iloc[:, int(n_col/2):])


Initial Dataframe Head:
         date  store_id store_type region    city  store_area_sqft product_id  \
0  2024-09-05  store_09          C   East  city_9             2287   prod_031   
1  2022-10-24  store_02          C   East  city_2             2627   prod_041   
2  2023-04-19  store_06          B   West  city_6             2547   prod_022   
3  2024-06-22  store_06          B   West  city_6             2547   prod_037   
4  2024-07-20  store_02          C   East  city_2             2627   prod_018   

   category  base_price  final_price  discount_pct  
0      Home       43.01        43.33           0.0  
1    Sports      121.31        86.51          30.0  
2  Clothing       11.10        11.05           0.0  
3    Beauty      272.28       284.06           0.0  
4      Home       56.11        56.58           0.0  
   promotion  is_holiday  day_of_week  weekend  units_sold  returns  \
0          0           0            3        0           8        1   
1          0           1    

In [244]:
# Check the raw data information before preprocessing.
print('\nInitial Dataframe Information:')
print(df.info())


Initial Dataframe Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164400 entries, 0 to 164399
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             164400 non-null  object 
 1   store_id         164400 non-null  object 
 2   store_type       164400 non-null  object 
 3   region           164400 non-null  object 
 4   city             164400 non-null  object 
 5   store_area_sqft  164400 non-null  int64  
 6   product_id       164400 non-null  object 
 7   category         164400 non-null  object 
 8   base_price       164400 non-null  float64
 9   final_price      164400 non-null  float64
 10  discount_pct     164400 non-null  float64
 11  promotion        164400 non-null  int64  
 12  is_holiday       164400 non-null  int64  
 13  day_of_week      164400 non-null  int64  
 14  weekend          164400 non-null  int64  
 15  units_sold       164400 non-null  int64  
 16  return

In [245]:
# Check the current data numeric stats before preprocessing.
print('\nInitial Dataframe Numeric Stats:')
print(df.describe().iloc[:, :int(n_col/3)])
print(df.describe().iloc[:, int(n_col/3):])


Initial Dataframe Numeric Stats:
       store_area_sqft     base_price    final_price   discount_pct  \
count    164400.000000  164400.000000  164400.000000  164400.000000   
mean       2440.700000      48.682199      46.781269       3.884519   
std         317.638196      49.139032      47.473471       7.073051   
min        1734.000000       5.800000       3.860000       0.000000   
25%        2287.000000      17.750000      16.880000       0.000000   
50%        2570.000000      31.030000      31.100000       0.000000   
75%        2611.000000      59.100000      56.670000       5.000000   
max        2904.000000     272.280000     293.690000      30.000000   

           promotion     is_holiday    day_of_week  
count  164400.000000  164400.000000  164400.000000  
mean        0.079453       0.013686       3.000000  
std         0.270445       0.116185       2.002286  
min         0.000000       0.000000       0.000000  
25%         0.000000       0.000000       1.000000  
50%     

As shown above, the data is already clean and ready to use.

# EDA (Exploratory Data Analysis)

## 1. Does the presence of holiday affect overall sales and revenue, both daily and monthly?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the ID columns in this section.
# Copy to ensure every change made in this section doesn't affect the main data.
df_1 = df[['date','is_holiday','net_units','net_revenue']].copy()
print(df_1.head())

         date  is_holiday  net_units  net_revenue
0  2024-09-05           0          7       303.32
1  2022-10-24           1          5       432.56
2  2023-04-19           0          2        22.09
3  2024-06-22           0          1       284.06
4  2024-07-20           0          2       113.16


### Holiday Effect Towards Daily Sales and Revenue

Sum sales (units) and revenue of all stores and products each day.

In [ ]:
df_1 = df_1.groupby(df_1['date']).sum()
# The line above sums all numeric data except `date` so `is_holiday` were also summed.
# However, `is_holiday` is conditional data and is better represented as binary (0 or 1) in a day-by-day data.
df_1.loc[df_1['is_holiday'] > 0, 'is_holiday'] = 1
print(df_1.head())

            is_holiday  net_units  net_revenue
date                                          
2022-01-01           0        633     32625.87
2022-01-02           0        636     31625.80
2022-01-03           0        574     30254.72
2022-01-04           0        532     26296.65
2022-01-05           0        568     25624.26


Find the correlation coefficients between variables (`is_holiday`, `net_units`, and `net_revenue`) to analyze the effect of holiday to daily sales and revenue.

In [248]:
corr_1_1 = df_1[['is_holiday', 'net_units', 'net_revenue']].corr()
print(corr_1_1)

             is_holiday  net_units  net_revenue
is_holiday     1.000000   0.169136     0.141445
net_units      0.169136   1.000000     0.919873
net_revenue    0.141445   0.919873     1.000000


Based on the results, the presence of holiday doesn't significantly affect the overall daily sales and revenue.

### Holiday Effect Towards Monthly Sales and Revenue

Every dates in the current data are already unique so we can erase the day in the `date` data to support monthly-based analysis process.

In [249]:
df_1.index = dtm(df_1.index)
print(df_1.head())

         is_holiday  net_units  net_revenue
date                                       
2022-01           0        633     32625.87
2022-01           0        636     31625.80
2022-01           0        574     30254.72
2022-01           0        532     26296.65
2022-01           0        568     25624.26


Group (sum) the numeric data based on `date` to earn monthly net sales and revenue.

In [250]:
df_1 = df_1.groupby(df_1.index).sum()
# In this section, `is_holiday` is no longer conditional and is expected to be summed as a representation for total holiday-days in a month.
print(df_1.head())

         is_holiday  net_units  net_revenue
date                                       
2022-01           1      18498    860972.53
2022-02           0      16240    743945.03
2022-03           1      18163    833257.90
2022-04           0      17575    830541.04
2022-05           0      18032    811823.17


Find the correlation coefficients between variables (`is_holiday`, `net_units`, and `net_revenue`).

In [251]:
corr_1_2 = df_1[['is_holiday', 'net_units', 'net_revenue']].corr()
print(corr_1_2)

             is_holiday  net_units  net_revenue
is_holiday     1.000000   0.218785     0.222019
net_units      0.218785   1.000000     0.994426
net_revenue    0.222019   0.994426     1.000000


Based on the results, the presence of holiday doesn't significantly affect the overall monthly sales and revenue. However, the effect shows more than the daily analysis in the previous section.

## 2. Is there any change in product's category trend during no-holiday months and holiday months?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the ID columns in this section.
# Copy to ensure every change made in this section doesn't affect the main data.
df_2 = df[['date','is_holiday','category','net_units']].copy()
print(df_2.head())

         date  is_holiday  category  net_units
0  2024-09-05           0      Home          7
1  2022-10-24           1    Sports          5
2  2023-04-19           0  Clothing          2
3  2024-06-22           0    Beauty          1
4  2024-07-20           0      Home          2


Every dates in the current data are already uniquely paired with each category so we can erase the day in the `date` data to support monthly-based analysis process.

In [253]:
df_2['date'] = dtm(df_2['date'])
print(df_2.head())

      date  is_holiday  category  net_units
0  2024-09           0      Home          7
1  2022-10           1    Sports          5
2  2023-04           0  Clothing          2
3  2024-06           0    Beauty          1
4  2024-07           0      Home          2


Count `net_units` by month and product's category while keeping the `is_holiday` properties.

In [ ]:
df_2 = df_2.groupby(['date', 'category'], as_index=False)[['is_holiday','net_units']].sum()
# The above line will sum `is_holiday` and `net_units` data based on unique pairs of `date` and `category`.
# However, in this section, we need `is_holiday` as a conditional data so it is better represented as binary (0 or 1).
df_2.loc[df_2['is_holiday'] > 0, 'is_holiday'] = 1
print(df_2.head())

      date     category  is_holiday  net_units
0  2022-01       Beauty           1       1670
1  2022-01     Clothing           1       6177
2  2022-01  Electronics           1       3936
3  2022-01         Home           1       5372
4  2022-01       Sports           1       1343


Only returns the highest sales product's category for each month.

In [255]:
df_2 = df_2.loc[df_2.groupby('date')['net_units'].idxmax()]
print(df_2.head())

       date  category  is_holiday  net_units
1   2022-01  Clothing           1       6177
6   2022-02  Clothing           0       5343
11  2022-03  Clothing           1       6203
16  2022-04  Clothing           0       6069
21  2022-05  Clothing           0       6079


Only returns the mode of the highest sales product's category across all of the no-holiday months and all of the holiday months.

In [256]:
df_2 = df_2.groupby('is_holiday')['category'].agg(lambda x: x.mode()[0])
print(df_2.head())

is_holiday
0    Clothing
1    Clothing
Name: category, dtype: object


As seen in the two latest dataframes, there's no change in product's category trend during no-holiday months and holiday months. Both product's category trends are clothing.

## 3. Does the weekend status affect overall daily sales and revenue?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the ID columns in this section.
# Copy to ensure every change made in this section doesn't affect the main data.
df_3 = df[['date','weekend','net_units','net_revenue']].copy()
print(df_3.head())

         date  weekend  net_units  net_revenue
0  2024-09-05        0          7       303.32
1  2022-10-24        0          5       432.56
2  2023-04-19        0          2        22.09
3  2024-06-22        1          1       284.06
4  2024-07-20        1          2       113.16


Sum sales (units) and revenue of all stores and products each day.

In [ ]:
df_3 = df_3.groupby(df_3['date']).sum()
# The line above sums all numeric data except `date` so `weekend` were also summed.
# However, `weekend` is conditional data and is better represented as binary (0 or 1) in a day-by-day data.
df_3.loc[df_3['weekend'] > 0, 'weekend'] = 1
print(df_3.head())

            weekend  net_units  net_revenue
date                                       
2022-01-01        1        633     32625.87
2022-01-02        1        636     31625.80
2022-01-03        0        574     30254.72
2022-01-04        0        532     26296.65
2022-01-05        0        568     25624.26


Find the correlation coefficients between variables (`weekend`, `net_units`, and `net_revenue`) to analyze the effect of weekend to daily sales and revenue.

In [259]:
corr_2 = df_3[['weekend', 'net_units', 'net_revenue']].corr()
print(corr_2)

              weekend  net_units  net_revenue
weekend      1.000000   0.511264     0.462136
net_units    0.511264   1.000000     0.919873
net_revenue  0.462136   0.919873     1.000000


Based on the results, the weekend status quite significantly affect the overall daily sales and revenue.

## 4. How is the overall day-by-day sales and revenue trend during a week?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the ID columns in this section.
# Because this section analyze day or `day_of_week` instead of `date`, we can also exclide the `date` column.
# Copy to ensure every change made in this section doesn't affect the main data.
df_4 = df[['day_of_week','net_units','net_revenue']].copy()
print(df_4.head())

   day_of_week  net_units  net_revenue
0            3          7       303.32
1            0          5       432.56
2            2          2        22.09
3            5          1       284.06
4            5          2       113.16


Find the average values of sales and revenue for each day of the week.

In [261]:
df_4 = df_4.groupby(df_4['day_of_week']).mean()
print(df_4)

             net_units  net_revenue
day_of_week                        
0             3.825563   177.054874
1             3.819278   175.540487
2             3.849615   176.279568
3             3.831197   175.841911
4             3.873120   178.832584
5             4.691125   214.380471
6             4.735329   219.188544


The results show that sales and revenue are higher during weekends. This shows a consistent result between this section and previous section, the weekend status quite significantly affect the overall daily sales and revenue.

## 5. Does the store type and area affect the customer experience, which lead to store's sales and revenue?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the `date` and ID columns in this section.
# Copy to ensure every change made in this section doesn't affect the main data.
df_5 = df[['store_type','store_area_sqft','avg_rating','net_units','net_revenue']].copy()
print(df_5.head())

  store_type  store_area_sqft  avg_rating  net_units  net_revenue
0          C             2287        3.41          7       303.32
1          C             2627        3.59          5       432.56
2          B             2547        3.86          2        22.09
3          B             2547        4.71          1       284.06
4          C             2627        3.86          2       113.16


Find the average values of customer experiences, sales, and revenue for each store type and area.

In [263]:
df_5 = df_5.groupby(['store_type', 'store_area_sqft'], as_index=False)[['avg_rating','net_units','net_revenue']].mean()
print(df_5)

  store_type  store_area_sqft  avg_rating  net_units  net_revenue
0          A             1734    3.972809   4.283577   196.241209
1          A             2593    3.964456   4.324088   200.346071
2          A             2596    3.968857   4.059367   186.223064
3          B             2547    3.965058   3.888564   175.711395
4          C             2055    3.968860   4.095438   190.132041
5          C             2287    3.968108   3.859124   178.184290
6          C             2453    3.968673   4.051886   186.245386
7          C             2611    3.970072   3.870073   177.580783
8          C             2627    3.965832   4.363808   202.385544
9          C             2904    3.974178   4.103771   188.853874


### Store Type Effect Towards Average Customer Experiences, Sales, and Revenue

Find the average values of customer experiences, sales, and revenue for each store type only.

In [264]:
df_5_type = df_5.drop(columns='store_area_sqft').copy()
df_5_type = df_5_type.groupby(df_5_type['store_type']).mean()
print(df_5_type)

            avg_rating  net_units  net_revenue
store_type                                    
A             3.968707   4.222344   194.270115
B             3.965058   3.888564   175.711395
C             3.969287   4.057350   187.230320


The result above shows that store type, though it doesn't significantly affect customer experiences, quite significantly affect store's sales and revenue. Store type C has the highest rank, but store type A has the highest net sales and revenue.

### Store Area (in sqft) Effect Towards Average Customer Experiences, Sales, and Revenue

Find the correlation coefficients between `store_area_sqft`, `avg_rating`, `net_units`, and `net_revenue`.

In [265]:
df_5_area = df_5[['store_area_sqft', 'avg_rating', 'net_units', 'net_revenue']].corr()
print(df_5_area)

                 store_area_sqft  avg_rating  net_units  net_revenue
store_area_sqft         1.000000   -0.174085  -0.132697    -0.125751
avg_rating             -0.174085    1.000000  -0.065062    -0.070525
net_units              -0.132697   -0.065062   1.000000     0.990896
net_revenue            -0.125751   -0.070525   0.990896     1.000000


The result above shows that store area doesn't significantly affect either customer experiences, sales, nor revenue. Interestingly, larger store areas show a negative correlation with ratings, sales, and revenue suggesting that bigger spaces might not necessarily improve customer satisfaction. In addition, surprisingly, customer experiences also doesn't significantly affect either sales nor revenue and is on negative correlation, which means a higher rating results to lower sales and revenue.

## 6. Which category of product is the most popular in each city month-by-month?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the `date` and ID columns in this section.
# Copy to ensure every change made in this section doesn't affect the main data.
df_6 = df[['date','city','category','net_units']].copy()
print(df_6.head())

         date    city  category  net_units
0  2024-09-05  city_9      Home          7
1  2022-10-24  city_2    Sports          5
2  2023-04-19  city_6  Clothing          2
3  2024-06-22  city_6    Beauty          1
4  2024-07-20  city_2      Home          2


Every dates in the current data are already uniquely paired with each category so we can erase the day in the `date` data to support monthly-based analysis process.

In [267]:
df_6['date'] = dtm(df_6['date'])
print(df_6)

           date    city  category  net_units
0       2024-09  city_9      Home          7
1       2022-10  city_2    Sports          5
2       2023-04  city_6  Clothing          2
3       2024-06  city_6    Beauty          1
4       2024-07  city_2      Home          2
...         ...     ...       ...        ...
164395  2024-03  city_2    Sports          6
164396  2023-11  city_3      Home          2
164397  2024-05  city_6      Home          4
164398  2024-09  city_2    Sports          1
164399  2024-03  city_1    Beauty          5

[164400 rows x 4 columns]


Count `net_units` by month and product's category while keeping the `city` data.

In [268]:
df_6 = df_6.groupby(['date', 'category','city'], as_index=False)[['net_units']].sum()
# The above line will sum `net_units` data based on unique pairs of `date`, `category`, and 'city'.
print(df_6.head())

      date category     city  net_units
0  2022-01   Beauty   city_1        176
1  2022-01   Beauty  city_10        151
2  2022-01   Beauty   city_2        150
3  2022-01   Beauty   city_3        163
4  2022-01   Beauty   city_4        181


Separate data by city to support city-based analysis process and to ensure every store carries equal weights.

In [269]:
city_dfs = {}
separate(df_6, city_dfs, 'city')

Returns the highest sales product's category for each pair of month and place.

In [270]:
for city, df_city in city_dfs.items():
    df_city = df_city.loc[df_city.groupby(['date'])['net_units'].idxmax()]
    city_dfs[city] = df_city
    print(city,':\n',df_city.head())

city_1 :
         date  category  net_units
144  2022-01  Clothing        647
122  2022-02  Clothing        579
175  2022-03  Clothing        697
40   2022-04  Clothing        625
85   2022-05  Clothing        571
city_10 :
         date  category  net_units
330  2022-01  Clothing        697
314  2022-02  Clothing        592
350  2022-03  Clothing        583
339  2022-04  Clothing        668
340  2022-05  Clothing        652
city_2 :
         date  category  net_units
462  2022-01  Clothing        678
430  2022-02  Clothing        566
526  2022-03  Clothing        644
417  2022-04  Clothing        670
362  2022-05  Clothing        732
city_3 :
         date  category  net_units
710  2022-01      Home        556
572  2022-02  Clothing        459
581  2022-03  Clothing        539
550  2022-04  Clothing        561
630  2022-05      Home        514
city_4 :
         date  category  net_units
760  2022-01  Clothing        613
731  2022-02  Clothing        523
868  2022-03  Clothing        6

To simplify the pattern analyzation process, we can group by continuous segments with the same category as below.

In [271]:
for city, df_city in city_dfs.items():
    # df_city = series_group(df_city)
    df_city = series_group(df_city, 'category', 'net_units')
    print(city,':\n',df_city)

city_1 :
                 date  category  total_item
0  2022-01 - 2023-10  Clothing       13692
1  2023-11 - 2023-12      Home        1635
2  2024-01 - 2024-07  Clothing        4166
3  2024-08 - 2024-08      Home         557
4  2024-09 - 2024-12  Clothing        2909
city_10 :
                 date  category  total_item
0  2022-01 - 2022-12  Clothing        7784
1  2023-01 - 2023-01      Home         636
2  2023-02 - 2024-06  Clothing       10503
3  2024-07 - 2024-07      Home         513
4  2024-08 - 2024-12  Clothing        3570
city_2 :
                 date  category  total_item
0  2022-01 - 2023-05  Clothing       11722
1  2023-06 - 2023-06      Home         546
2  2023-07 - 2023-08  Clothing        1194
3  2023-09 - 2023-09      Home         660
4  2023-10 - 2024-11  Clothing        9768
5  2024-12 - 2024-12      Home         947
city_3 :
                 date  category  total_item
0  2022-01 - 2022-01      Home         556
1  2022-02 - 2022-04  Clothing        1559
2  2022-05 - 

As shown above, each city displays different trends and preferences over time. However, the top product's category across those months and cities are always whether **Clothing** or **Home**.

## 7. How does discount percentages on products affect store's sales and revenue?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the `date` in this section.
# Discount percentage applies to one specific product and the product is not always on discount.
# Therefore, we need to include product ID data and analyze the effect for each product
# Copy to ensure every change made in this section doesn't affect the main data.
df_7 = df[['product_id','discount_pct','net_units','net_revenue']].copy()
print(df_7.head())

  product_id  discount_pct  net_units  net_revenue
0   prod_031           0.0          7       303.32
1   prod_041          30.0          5       432.56
2   prod_022           0.0          2        22.09
3   prod_037           0.0          1       284.06
4   prod_018           0.0          2       113.16


Separate data by product ID to support product-based analysis process and to ensure every product carries equal weights.

In [219]:
disc_dfs = {}
separate(df_7, disc_dfs, 'product_id')

In [220]:
# Output examples.
print('prod_031:\n',disc_dfs['prod_031'].head())
print('prod_041:\n',disc_dfs['prod_041'].head())
print('prod_022:\n',disc_dfs['prod_022'].head())

prod_031:
        discount_pct  net_units  net_revenue
98618           0.0          1        44.42
98619           0.0          5       214.38
98620           0.0          4       173.08
98621           0.0         10       429.38
98622           0.0          4       177.32
prod_041:
         discount_pct  net_units  net_revenue
131632          30.0          4       344.74
131633           0.0          3       364.89
131634           0.0          3       366.98
131635           0.0          2       247.80
131636          15.0          3       314.60
prod_022:
        discount_pct  net_units  net_revenue
68855           0.0          6        66.35
68856           0.0          4        44.94
68857           5.0          1        10.55
68858           0.0          3        35.00
68859           0.0          6        66.67


Find the correlation coefficients between `discount_pct`, `net_units`, and `net_revenue` for each product.

In [221]:
for prod_id, df_disc in disc_dfs.items():
    df_disc = df_disc[['discount_pct', 'net_units', 'net_revenue']].corr()
    disc_dfs[prod_id] = df_disc
    # print('\n',prod_id,':\n',disc_dfs[prod_id])

Average correlation values to find general insights.

In [222]:
disc_mat = []

avg_disc_df = avg_corr(prod_id, df_disc, disc_dfs, disc_mat)
print(avg_disc_df)

              discount_pct  net_units  net_revenue
discount_pct      1.000000   0.029940    -0.102719
net_units         0.029940   1.000000     0.987515
net_revenue      -0.102719   0.987515     1.000000


As shown above, discount percentages slightly impact sales and revenue. The higher the discount percentages, the higher the sales is. However, it is inversely proportional to the revenue.

## 8. How does product's promotion affect store's sales and revenue?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the `date` in this section.
# Promotion applies to one specific product and the product is not always on promotion.
# Therefore, we need to include product ID data and analyze the effect for each product
# Copy to ensure every change made in this section doesn't affect the main data.
df_8 = df[['product_id','promotion','net_units','net_revenue']].copy()
print(df_8.head())

  product_id  promotion  net_units  net_revenue
0   prod_031          0          7       303.32
1   prod_041          0          5       432.56
2   prod_022          0          2        22.09
3   prod_037          0          1       284.06
4   prod_018          0          2       113.16


Separate data by product ID to support product-based analysis process and to ensure every product carries equal weights.

In [224]:
promo_dfs = {}
separate(df_8, promo_dfs, 'product_id')

In [225]:
# Output examples.
print('prod_031:\n',promo_dfs['prod_031'].head())
print('prod_041:\n',promo_dfs['prod_041'].head())
print('prod_022:\n',promo_dfs['prod_022'].head())

prod_031:
        promotion  net_units  net_revenue
98618          0          1        44.42
98619          0          5       214.38
98620          0          4       173.08
98621          1         10       429.38
98622          0          4       177.32
prod_041:
         promotion  net_units  net_revenue
131632          0          4       344.74
131633          0          3       364.89
131634          0          3       366.98
131635          0          2       247.80
131636          0          3       314.60
prod_022:
        promotion  net_units  net_revenue
68855          0          6        66.35
68856          0          4        44.94
68857          0          1        10.55
68858          0          3        35.00
68859          0          6        66.67


Find the correlation coefficients between `promotion`, `net_units`, and `net_revenue` for each product.

In [226]:
for prod_id, df_promo in promo_dfs.items():
    df_promo = df_promo[['promotion', 'net_units', 'net_revenue']].corr()
    promo_dfs[prod_id] = df_promo
    # print(prod_id,':\n',df_promo)

Average correlation values to find general insights.

In [227]:
promo_mat = []

avg_promo_df = avg_corr(prod_id, df_promo, promo_dfs, promo_mat)
print(avg_promo_df)

             promotion  net_units  net_revenue
promotion     1.000000   0.304327     0.302372
net_units     0.304327   1.000000     0.987515
net_revenue   0.302372   0.987515     1.000000


As shown above, promotion affects sales and revenue. The impact is far greater than the impact of discount percentages. Moreover, promotion has directly proportional relations with both sales and revenue. The presence of promotion triggers higher sales and revenue.

## 9. How does the combination of discount and promotion give different effect to store's sales and revenue?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the `date` in this section.
# Promotion applies to one specific product and the product is not always on promotion.
# Therefore, we need to include product ID data and analyze the effect for each product
# Copy to ensure every change made in this section doesn't affect the main data.
df_9 = df[['product_id','discount_pct','promotion','net_units','net_revenue']].copy()
print(df_9.head())

  product_id  discount_pct  promotion  net_units  net_revenue
0   prod_031           0.0          0          7       303.32
1   prod_041          30.0          0          5       432.56
2   prod_022           0.0          0          2        22.09
3   prod_037           0.0          0          1       284.06
4   prod_018           0.0          0          2       113.16


To analyze how the combination of discount and promotion gives different effect to store's sales and revenue, we need to pair each discount and promotion in categories.

From the data preprocessing section, we already knew that the discount percentage value ranges from 0-30%. We will group these value into four categories: `no_disc`, `low_disc`, `mid_disc`, and `high_disc`. Meanwhile, the promotion value represents in a binary condition so there are only two categories: `no_promo` and `promo`.

In [229]:
df_9['disc_group'] = pd.cut(df_9['discount_pct'], bins=[-1, 0, 10, 20, 30], labels=['no_disc', 'low_disc', 'mid_disc', 'high_disc'])
df_9['promo_group'] = df_9['promotion'].map({0: 'no_promo', 1: 'promo'})
print(df_9.head())

  product_id  discount_pct  promotion  net_units  net_revenue disc_group  \
0   prod_031           0.0          0          7       303.32    no_disc   
1   prod_041          30.0          0          5       432.56  high_disc   
2   prod_022           0.0          0          2        22.09    no_disc   
3   prod_037           0.0          0          1       284.06    no_disc   
4   prod_018           0.0          0          2       113.16    no_disc   

  promo_group  
0    no_promo  
1    no_promo  
2    no_promo  
3    no_promo  
4    no_promo  


Separate data by product ID to support product-based analysis process and to ensure every product carries equal weights.

In [230]:
comb_dfs = {}
separate(df_9, comb_dfs, 'product_id')

for prod_id, df_comb in comb_dfs.items():
    df_comb = df_comb.drop(columns=['discount_pct','promotion'])
    # We will do a cross group analysis process for each product, averaging values of sales and revenue for each combination of discount and promotion.
    df_comb = df_comb.groupby(['disc_group', 'promo_group'], observed=True)[['net_units', 'net_revenue']].mean()
    comb_dfs[prod_id] = df_comb

In [231]:
# Output examples.
print('prod_031:\n',comb_dfs['prod_031'])
print('prod_041:\n',comb_dfs['prod_041'])
print('prod_022:\n',comb_dfs['prod_022'])

prod_031:
                         net_units  net_revenue
disc_group promo_group                        
no_disc    no_promo      3.617972   155.481581
           promo         5.837563   250.939492
low_disc   no_promo      3.478261   138.387022
           promo         5.463415   218.210732
mid_disc   no_promo      3.658470   129.864672
           promo         6.742857   235.990286
high_disc  no_promo      3.779661   114.377119
           promo         6.750000   204.708750
prod_041:
                         net_units  net_revenue
disc_group promo_group                        
no_disc    no_promo      2.678134   324.856023
           promo         4.502674   547.052620
low_disc   no_promo      2.902597   325.389199
           promo         5.000000   563.991778
mid_disc   no_promo      3.042373   304.422119
           promo         4.653846   469.412692
high_disc  no_promo      3.412500   289.081125
           promo         5.400000   455.961000
prod_022:
                         net

Average correlation values to find general insights.

In [232]:
comb_mat = []

avg_comb_df = cross_avg_mean(prod_id, df_comb, comb_dfs, comb_mat)
print(avg_comb_df)

                        net_units  net_revenue
disc_group promo_group                        
no_disc    no_promo      3.849325   183.086158
           promo         6.482037   309.452271
low_disc   no_promo      3.896958   175.041456
           promo         6.449941   289.815817
mid_disc   no_promo      4.009372   162.782706
           promo         6.703554   270.579875
high_disc  no_promo      4.184417   148.733762
           promo         6.813056   249.538637


As shown above, combination of discount and promotion gives different effect to sales and revenue. High discount with promotion results in highest sales, but no discount with promotion results in highest revenue. This affirm the two previous sections' result that promotion has greater positive impact to sales and revenue.

## 10. Does online transaction affect the customer experience (returns and rating)?

Get the necessary columns from the preprocessed data.

In [ ]:
# Each row is confirmed unique from the previous step so we can exclude the `date` in this section.
# Customer experience varies between stores.
# Therefore, we need to include store ID data to analyze online transaction effect to customer experience.
# Copy to ensure every change made in this section doesn't affect the main data.
df_10 = df[['store_id', 'online','returns','avg_rating']].copy()
print(df_10.head())

   store_id  online  returns  avg_rating
0  store_09       1        1        3.41
1  store_02       0        0        3.59
2  store_06       0        0        3.86
3  store_06       0        1        4.71
4  store_02       0        0        3.86


Separate data by store ID to support store-based analysis process and to ensure every store carries equal weights.

In [234]:
cx_dfs = {}
separate(df_10, cx_dfs, 'store_id')

In [235]:
# Output examples.
print('store_09:\n',cx_dfs['store_09'].head())
print('store_02:\n',cx_dfs['store_02'].head())
print('store_06:\n',cx_dfs['store_06'].head())

store_09:
         online  returns  avg_rating
131520       1        0        4.27
131521       0        0        4.37
131522       0        0        4.57
131523       0        0        3.88
131524       0        0        3.46
store_02:
        online  returns  avg_rating
16440       0        0        4.34
16441       1        0        4.25
16442       0        0        4.60
16443       1        0        4.23
16444       1        0        4.31
store_06:
        online  returns  avg_rating
82200       0        0        4.29
82201       1        0        4.05
82202       0        0        4.06
82203       0        0        4.42
82204       0        0        3.23


Find the correlation coefficients between `online`, `returns`, and `avg_rating` for each store.

In [ ]:
for store_id, df_cx in cx_dfs.items():
    df_cx = df_cx[['online', 'returns', 'avg_rating']].corr()
    cx_dfs[store_id] = df_cx
    print(store_id,':\n',df_cx)

store_01 :
               online   returns  avg_rating
online      1.000000  0.007629    0.008385
returns     0.007629  1.000000    0.005793
avg_rating  0.008385  0.005793    1.000000
store_02 :
               online   returns  avg_rating
online      1.000000 -0.000830    0.008339
returns    -0.000830  1.000000    0.003312
avg_rating  0.008339  0.003312    1.000000
store_03 :
               online   returns  avg_rating
online      1.000000 -0.012437    0.003949
returns    -0.012437  1.000000   -0.001758
avg_rating  0.003949 -0.001758    1.000000
store_04 :
               online   returns  avg_rating
online      1.000000  0.001898    0.007897
returns     0.001898  1.000000   -0.005092
avg_rating  0.007897 -0.005092    1.000000
store_05 :
               online   returns  avg_rating
online      1.000000  0.009061    0.003746
returns     0.009061  1.000000    0.000032
avg_rating  0.003746  0.000032    1.000000
store_06 :
               online   returns  avg_rating
online      1.000000 -0.0

Average correlation values to find general insights.

In [237]:
cx_mat = []

avg_cx_df = avg_corr(store_id, df_cx, cx_dfs, cx_mat)
print(avg_cx_df)

              online   returns  avg_rating
online      1.000000  0.000254    0.005357
returns     0.000254  1.000000    0.000790
avg_rating  0.005357  0.000790    1.000000


As shown above, the correlation values is too low so we can say online transaction doesn't affect customer experience.